In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('fraudTest.csv', index_col=0)
data.drop(columns=[
    "cc_num", "first", "last", "street",
    "trans_num", "trans_date_trans_time",
    "job", "merchant", "city", "zip"
],
          inplace=True)

In [3]:
data.head()

## Now we proceed to transform categorical variables

,category,amt,gender,state,lat,long,city_pop,dob,unix_time,merch_lat,merch_long,is_fraud
0,personal_care,2.86,M,SC,33.9659,-80.9355,333497,1968-03-19,1371816865,33.986391,-81.200714,0
1,personal_care,29.84,F,UT,40.3207,-110.4360,302,1990-01-17,1371816873,39.450498,-109.960431,0
2,health_fitness,41.28,F,NY,40.6729,-73.5365,34496,1970-10-21,1371816893,40.495810,-74.196111,0
3,misc_pos,60.05,M,FL,28.5697,-80.8191,54767,1987-07-25,1371816915,28.812398,-80.883061,0
4,travel,3.19,M,MI,44.2529,-85.0170,1126,1955-07-06,1371816917,44.959148,-85.884734,0


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

gender_le = LabelEncoder()

data['gender'] = gender_le.fit_transform(data['gender'])

category_ohe = OneHotEncoder(drop='first', sparse_output=False)

cols_drop = ['category', 'state']

category_encoded = category_ohe.fit_transform(data[cols_drop])

category_df = pd.DataFrame(category_encoded,
                           columns = category_ohe.get_feature_names_out(cols_drop),
                           index = data.index)

data = pd.concat([data.drop(columns=cols_drop), category_df],
                 axis = 1)

data.head()

X = data.drop(columns=['is_fraud'])
y = data['is_fraud']

## Now the data set is properly encoded and ready for preprocessing

In [5]:
from matplotlib import scale
from sklearn.model_selection import train_test_split

scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=100)
## Stratify makes sure both train and test have the same proportion of the dependent categorical value.

num_cols = ['amt','lat','long','city_pop','unix_time','merch_lat','merch_long']

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.fit_transform(X_test[num_cols])

In [8]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping